## 확률적 경사 하강법을 이용한 행렬 분해
P와 Q 행렬로 계산된 예측 R 행렬 값이 실제 R 행렬 값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용 함수 최적화를 통해 P와 Q를 유추해내는 것

In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
            [np.NaN, 5, np.NaN, 3, 1],
            [np.NaN, np.NaN, 3, 4, 4],
            [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K = 3

# P와 Q 행렬의 크기 지정 후 정규 분포를 가진 임의의 값으로 입력
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [6]:
# 실제 R 행렬과 예측 행렬의 오차를 구하는 함수 만들기
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [9]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i,j,r in non_zeros:
        # 실제값과 예측값의 차이인 오류 값 구함
        eij = r - np.dot(P[i,:], Q[j,:].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij*Q[j,:] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij*P[i,:] - r_lambda*Q[j,:])
        
        rmse = get_rmse(R, P, Q, non_zeros)
        if (steps % 50) == 0:
            print('### iteration step: ', step, 'rmse: ', rmse)

### iteration step:  0 rmse:  3.261355059488935
### iteration step:  0 rmse:  3.26040057174686
### iteration step:  0 rmse:  3.253984404542389
### iteration step:  0 rmse:  3.2521583839863624
### iteration step:  0 rmse:  3.252335303789125
### iteration step:  0 rmse:  3.251072196430487
### iteration step:  0 rmse:  3.2492449982564864
### iteration step:  0 rmse:  3.247416477570409
### iteration step:  0 rmse:  3.241926055455223
### iteration step:  0 rmse:  3.2400454107613084
### iteration step:  0 rmse:  3.240166740749792
### iteration step:  0 rmse:  3.2388050277987723
### iteration step:  1 rmse:  3.2370456458092303
### iteration step:  1 rmse:  3.236116990209145
### iteration step:  1 rmse:  3.2295142762113724
### iteration step:  1 rmse:  3.2276592778943614
### iteration step:  1 rmse:  3.227834708372119
### iteration step:  1 rmse:  3.226427324850506
### iteration step:  1 rmse:  3.2243158226413944
### iteration step:  1 rmse:  3.2222740490081994
### iteration step:  1 rmse:  3.

### iteration step:  23 rmse:  1.970490293679612
### iteration step:  23 rmse:  1.9627704066508864
### iteration step:  23 rmse:  1.9503587053704232
### iteration step:  23 rmse:  1.9383304866849076
### iteration step:  23 rmse:  1.91649949348962
### iteration step:  23 rmse:  1.9156320275778382
### iteration step:  23 rmse:  1.9152750494266881
### iteration step:  23 rmse:  1.9134072421623636
### iteration step:  24 rmse:  1.9001678612205215
### iteration step:  24 rmse:  1.8973198499232757
### iteration step:  24 rmse:  1.8840581089904715
### iteration step:  24 rmse:  1.8763337590921114
### iteration step:  24 rmse:  1.8746992162016969
### iteration step:  24 rmse:  1.8670644686404474
### iteration step:  24 rmse:  1.8554751233290963
### iteration step:  24 rmse:  1.8435627055856902
### iteration step:  24 rmse:  1.8202803180124403
### iteration step:  24 rmse:  1.8197744472659072
### iteration step:  24 rmse:  1.8195274883003187
### iteration step:  24 rmse:  1.8182944911709282
###

### iteration step:  46 rmse:  0.5724490708083293
### iteration step:  46 rmse:  0.5718053656805689
### iteration step:  47 rmse:  0.5632359840574156
### iteration step:  47 rmse:  0.5635045073068867
### iteration step:  47 rmse:  0.5620039024747391
### iteration step:  47 rmse:  0.5618580349069338
### iteration step:  47 rmse:  0.5619703301430213
### iteration step:  47 rmse:  0.5611202572740296
### iteration step:  47 rmse:  0.5611416393704858
### iteration step:  47 rmse:  0.5602764042184408
### iteration step:  47 rmse:  0.5500591024309135
### iteration step:  47 rmse:  0.5492805883484339
### iteration step:  47 rmse:  0.5491179155199218
### iteration step:  47 rmse:  0.5483476774942395
### iteration step:  48 rmse:  0.5406683572766874
### iteration step:  48 rmse:  0.540781711512994
### iteration step:  48 rmse:  0.5394057291732345
### iteration step:  48 rmse:  0.539266725970842
### iteration step:  48 rmse:  0.5393779559134593
### iteration step:  48 rmse:  0.5386147029059574
##

### iteration step:  62 rmse:  0.34364952658728404
### iteration step:  62 rmse:  0.3436688756263796
### iteration step:  62 rmse:  0.3406761473420774
### iteration step:  62 rmse:  0.3396377763810412
### iteration step:  62 rmse:  0.33902701830576476
### iteration step:  62 rmse:  0.338020400134645
### iteration step:  63 rmse:  0.3368889860650682
### iteration step:  63 rmse:  0.33610862421101717
### iteration step:  63 rmse:  0.33563856546795096
### iteration step:  63 rmse:  0.33517395798677885
### iteration step:  63 rmse:  0.33519898871774734
### iteration step:  63 rmse:  0.334996861428818
### iteration step:  63 rmse:  0.3349255482884708
### iteration step:  63 rmse:  0.3349545186260696
### iteration step:  63 rmse:  0.33216029561474
### iteration step:  63 rmse:  0.33115384873612996
### iteration step:  63 rmse:  0.3305440254329402
### iteration step:  63 rmse:  0.3295619326508827
### iteration step:  64 rmse:  0.32856835984280097
### iteration step:  64 rmse:  0.3277896041855

### iteration step:  84 rmse:  0.21222979959118926
### iteration step:  84 rmse:  0.21224041465299343
### iteration step:  84 rmse:  0.21218106951728724
### iteration step:  84 rmse:  0.2117942778669273
### iteration step:  84 rmse:  0.21184366002251603
### iteration step:  84 rmse:  0.21081724310080024
### iteration step:  84 rmse:  0.21044201132242257
### iteration step:  84 rmse:  0.21006060925884498
### iteration step:  84 rmse:  0.20944553919184322
### iteration step:  85 rmse:  0.20933706655031922
### iteration step:  85 rmse:  0.2088503387307898
### iteration step:  85 rmse:  0.2087130445430204
### iteration step:  85 rmse:  0.20816330420741858
### iteration step:  85 rmse:  0.2081746835269492
### iteration step:  85 rmse:  0.20811850975019042
### iteration step:  85 rmse:  0.2077266152662635
### iteration step:  85 rmse:  0.20777548372239588
### iteration step:  85 rmse:  0.20678158574637448
### iteration step:  85 rmse:  0.20642654553865802
### iteration step:  85 rmse:  0.206

...

### iteration step:  900 rmse:  0.016470695682261876
### iteration step:  900 rmse:  0.016398314989165292
### iteration step:  900 rmse:  0.016512806333073466
### iteration step:  900 rmse:  0.01652811087350182
### iteration step:  900 rmse:  0.016351122754892394
### iteration step:  900 rmse:  0.016328629783842166
### iteration step:  900 rmse:  0.016007096878603234
### iteration step:  900 rmse:  0.016141544071514122
### iteration step:  900 rmse:  0.016226430843994055
### iteration step:  900 rmse:  0.01648146573819501
### iteration step:  901 rmse:  0.016520638490660936
### iteration step:  901 rmse:  0.01661194310824155
### iteration step:  901 rmse:  0.01646999863446128
### iteration step:  901 rmse:  0.016397639800331335
### iteration step:  901 rmse:  0.0165121364390384
### iteration step:  901 rmse:  0.0165274346952435
### iteration step:  901 rmse:  0.01635046868677171
### iteration step:  901 rmse:  0.016327968230116945
### iteration step:  901 rmse:  0.016006412525942003
##

### iteration step:  925 rmse:  0.015989774641360126
### iteration step:  925 rmse:  0.01612452674896256
### iteration step:  925 rmse:  0.016209617300311268
### iteration step:  925 rmse:  0.0164645698816159
### iteration step:  926 rmse:  0.01650429493688661
### iteration step:  926 rmse:  0.016594671393689798
### iteration step:  926 rmse:  0.016452329103019102
### iteration step:  926 rmse:  0.016380515310711295
### iteration step:  926 rmse:  0.016495144074508146
### iteration step:  926 rmse:  0.01651029267375997
### iteration step:  926 rmse:  0.016333849215129665
### iteration step:  926 rmse:  0.01631116684113934
### iteration step:  926 rmse:  0.015989072493869473
### iteration step:  926 rmse:  0.01612383672749314
### iteration step:  926 rmse:  0.01620893537862399
### iteration step:  926 rmse:  0.01646388350456253
### iteration step:  927 rmse:  0.016503629487192154
### iteration step:  927 rmse:  0.016593970765622027
### iteration step:  927 rmse:  0.016451612597837298
##

### iteration step:  939 rmse:  0.016585505231535076
### iteration step:  939 rmse:  0.016442956702550273
### iteration step:  939 rmse:  0.01637142539126979
### iteration step:  939 rmse:  0.01648612273494387
### iteration step:  939 rmse:  0.01650119872662136
### iteration step:  939 rmse:  0.01632500545884842
### iteration step:  939 rmse:  0.016302232222864792
### iteration step:  939 rmse:  0.01597988017310171
### iteration step:  939 rmse:  0.016114801620794546
### iteration step:  939 rmse:  0.01620000528090383
### iteration step:  939 rmse:  0.016454887755400115
### iteration step:  940 rmse:  0.01649489809545325
### iteration step:  940 rmse:  0.016584794965821274
### iteration step:  940 rmse:  0.01644223058584
### iteration step:  940 rmse:  0.016370720980628802
### iteration step:  940 rmse:  0.016485423597592294
### iteration step:  940 rmse:  0.01650049414021074
### iteration step:  940 rmse:  0.016324319545595923
### iteration step:  940 rmse:  0.016301539419771298
### i

### iteration step:  964 rmse:  0.01646842665055811
### iteration step:  964 rmse:  0.016483371825224715
### iteration step:  964 rmse:  0.016307622313091328
### iteration step:  964 rmse:  0.01628468085404587
### iteration step:  964 rmse:  0.015961872260557366
### iteration step:  964 rmse:  0.01609709411154927
### iteration step:  964 rmse:  0.01618249849826448
### iteration step:  964 rmse:  0.01643721639657487
### iteration step:  965 rmse:  0.0164776971283408
### iteration step:  965 rmse:  0.016566803228365148
### iteration step:  965 rmse:  0.016423843057235838
### iteration step:  965 rmse:  0.016352875135364316
### iteration step:  965 rmse:  0.016467709538677183
### iteration step:  965 rmse:  0.01648264970389976
### iteration step:  965 rmse:  0.01630691698142022
### iteration step:  965 rmse:  0.016283968961179177
### iteration step:  965 rmse:  0.015961143112149186
### iteration step:  965 rmse:  0.016096376932965603
### iteration step:  965 rmse:  0.016181789322278137
##

### iteration step:  988 rmse:  0.01646585643414301
### iteration step:  988 rmse:  0.01629049101197331
### iteration step:  988 rmse:  0.016267395386277497
### iteration step:  988 rmse:  0.015944192500112152
### iteration step:  988 rmse:  0.016079700776399898
### iteration step:  988 rmse:  0.016165296801000567
### iteration step:  988 rmse:  0.016419813080533904
### iteration step:  989 rmse:  0.016460702367747528
### iteration step:  989 rmse:  0.016549121890007085
### iteration step:  989 rmse:  0.01640578240516234
### iteration step:  989 rmse:  0.01633533291746704
### iteration step:  989 rmse:  0.01645029383957075
### iteration step:  989 rmse:  0.016465118462721758
### iteration step:  989 rmse:  0.016289768221821203
### iteration step:  989 rmse:  0.016266666316665207
### iteration step:  989 rmse:  0.015943447868686806
### iteration step:  989 rmse:  0.01607896804314118
### iteration step:  989 rmse:  0.016164572035828186
### iteration step:  989 rmse:  0.0164190790493453
#

In [10]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
